In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image

In [2]:
data_dir = r'D:/downloads/fire/fire'

In [3]:
def check_corrupt_images(directory):
    corrupt_images = []
    
    for root, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                with Image.open(file_path) as img:
                    img.load()  # Fully load image
                    img.convert("RGB")  # Ensure format compatibility
            except (IOError, SyntaxError, OSError, AttributeError) as e:
                corrupt_images.append(file_path)
                print(f"❌ Corrupt image detected: {file_path} - {e}")

    # 🗑️ Delete corrupted images
    for img in corrupt_images:
        try:
            os.remove(img)
            print(f"✅ Deleted: {img}")
        except Exception as e:
            print(f"⚠️ Failed to delete {img}: {e}")

# 🔹 Run before training
check_corrupt_images(data_dir)

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% validation split
)

In [5]:
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)


Found 2805 images belonging to 2 classes.


In [6]:
validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)


Found 700 images belonging to 2 classes.


In [7]:
input_layer = layers.Input(shape=(224, 224, 3))

In [8]:
base_model = keras.applications.MobileNetV2(
    input_shape=(224, 224, 3), include_top=False, weights='imagenet'
)
base_model.trainable = False

In [9]:
x = base_model(input_layer, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)
output_layer = layers.Dense(1, activation='sigmoid')(x)

In [10]:
model = keras.Model(inputs=input_layer, outputs=output_layer)


In [11]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [12]:
epochs = 15
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=epochs
)


Epoch 1/15


C:\Users\HP\python app\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


88/88 ━━━━━━━━━━━━━━━━━━━━ 172s 2s/step - accuracy: 0.9154 - loss: 0.3327 - val_accuracy: 0.9586 - val_loss: 0.1517
Epoch 2/15
88/88 ━━━━━━━━━━━━━━━━━━━━ 105s 1s/step - accuracy: 0.9616 - loss: 0.1206 - val_accuracy: 0.9286 - val_loss: 0.1821
Epoch 3/15
88/88 ━━━━━━━━━━━━━━━━━━━━ 94s 1s/step - accuracy: 0.9621 - loss: 0.1305 - val_accuracy: 0.9186 - val_loss: 0.2070
Epoch 4/15
88/88 ━━━━━━━━━━━━━━━━━━━━ 94s 1s/step - accuracy: 0.9619 - loss: 0.1075 - val_accuracy: 0.9186 - val_loss: 0.1953
Epoch 5/15
88/88 ━━━━━━━━━━━━━━━━━━━━ 94s 1s/step - accuracy: 0.9643 - loss: 0.1029 - val_accuracy: 0.9257 - val_loss: 0.1933
Epoch 6/15
88/88 ━━━━━━━━━━━━━━━━━━━━ 115s 1s/step - accuracy: 0.9618 - loss: 0.1119 - val_accuracy: 0.9429 - val_loss: 0.1623
Epoch 7/15
88/88 ━━━━━━━━━━━━━━━━━━━━ 96s 1s/step - accuracy: 0.9566 - loss: 0.1288 - val_accuracy: 0.9071 - val_loss: 0.2063
Epoch 8/15
88/88 ━━━━━━━━━━━━━━━━━━━━ 95s 1s/step - accuracy: 0.9611 - loss: 0.0961 - val_accuracy: 0.9186 - val_loss: 0.2121


In [13]:
loss, accuracy = model.evaluate(validation_generator)
print(f"✅ Validation Loss: {loss:.4f}")
print(f"✅ Validation Accuracy: {accuracy:.4f}")

22/22 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.8923 - loss: 0.2489
✅ Validation Loss: 0.2288
✅ Validation Accuracy: 0.8986


In [14]:
model.save("fire_detection.keras", save_format="keras")

In [16]:
loaded_model = keras.models.load_model("fire_detection.keras")
print("📌 Model Input Shape After Loading:", loaded_model.input_shape)


📌 Model Input Shape After Loading: (None, 224, 224, 3)


In [17]:
def safe_load_img(img_path, target_size):
    try:
        img = image.load_img(img_path, target_size=target_size)
        return img.convert("RGB")  # Ensure compatibility
    except Exception as e:
        print(f"⚠️ Skipping unreadable image: {img_path}. Error: {e}")
        return None

In [18]:
def predict_fire(img_path):
    img = safe_load_img(img_path, target_size=(224, 224))
    if img is None:
        return  # Skip prediction if image is unreadable

    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Convert to batch format
    img_array /= 255.0  # Normalize

    prediction = model.predict(img_array)[0][0]  # Get prediction value
    print(f"🔍 Prediction Confidence: {prediction:.4f}")
    
    if prediction > 0.5:
        print("🔥 Fire detected!")
    else:
        print("✅ No Fire detected.")

# ✅ Test Prediction
test_image_path = "D:/downloads/fire/fire/val/images/1a228a4049efc30e.jpg"
predict_fire(test_image_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 873ms/step
🔍 Prediction Confidence: 0.7553
🔥 Fire detected!
